In [50]:
import time 

# Importing the libraries
import configparser
import tweepy
import pandas as pd

import uuid

from datetime import datetime
data_ref=datetime.today().strftime("%Y-%m-%d")

# Read the config file
config = configparser.ConfigParser()
config.read('../config.ini_local')

# Read the values
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']


# Authenticate
auth = tweepy.OAuth1UserHandler(
   api_key, api_key_secret, access_token, access_token_secret
)

api = tweepy.API(auth)

In [51]:
# Import libraries
import glob
import pandas as pd

In [52]:
path = "../data/influencer_tweets/"

csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df   = pd.concat(df_list, ignore_index=True)

In [53]:
df.shape

(112200, 6)

In [54]:
df.head(2)

,id,userID,created_at,full_text,favorite_count,retweet_count
0,1644820236749635591,BRKsEDU,2023-04-08 21:51:18+00:00,@MarioThael Muito obrigado!!! Quem editou foi ...,4,1
1,1644800548812587008,BRKsEDU,2023-04-08 20:33:04+00:00,@V_JuniorR Valeu,1,0


In [55]:
df2 = df.drop_duplicates(subset='id', keep="last")
df2.shape

(567, 6)

In [56]:
df2.userID.value_counts()

Loiranoob    204
BRKsEDU      187
Cellbit      176
Name: userID, dtype: int64

In [57]:
coletar_iteracoes = df2.id.to_list()

In [58]:
len(coletar_iteracoes)

567

In [59]:
# Iterar sobre os tweets e imprimir o texto de cada um
def get_text_retweet(api, post_id):

    # Coletar a publicação
    post = api.get_status(post_id)

    # Fazer uma busca por tweets que sejam respostas à publicação
    replies = tweepy.Cursor(api.search_tweets, q="to:{}".format(post.author.screen_name),
                            since_id=post.id, tweet_mode="extended").items()

    lista_replies = []
    lista_tweet = []
    uui_id = str(uuid.uuid4())
    for tweet in replies:
        if hasattr(tweet, "in_reply_to_status_id_str"):
            if tweet.in_reply_to_status_id_str == post.id_str:
                # print(tweet.full_text)
                lista_replies.append(tweet)
                lista_tweet.append({
                    "full_text": tweet.full_text,
                    "created_at": tweet.created_at,
                    "id":tweet.id_str 
                })
    if len(lista_tweet) > 0:
        d = pd.DataFrame(lista_tweet).to_csv(f'../data/tweet_iteracoes/{post_id}_{data_ref}_{uui_id}.csv', index=False)

In [60]:
path = "../data/tweet_iteracoes/"

coletados_tweets = glob.glob(path + "/*.csv")

In [61]:
import re 
# define the pattern to match
pattern = r'tweet_iteracoes\/(\d+)_\d{4}-\d{2}-\d{2}'

# initialize an empty list to store the tweet IDs
tweet_ids = []

# iterate over each file path
for file_path in coletados_tweets:
    # search for the pattern in the file path
    match = re.search(pattern, file_path)
    # if a match is found, extract the ID and add it to the list
    if match:
        tweet_id = match.group(1)
        tweet_ids.append(int(tweet_id))


In [62]:
len(coletar_iteracoes), len(tweet_ids)

(567, 8)

## este codigo auxiliar a remover coletas ja feitas

In [63]:
result = list(set(coletar_iteracoes) - set(tweet_ids))
len(result)

559

In [64]:
from tqdm import tqdm
for data_loop in tqdm(result):
    get_text_retweet(api, data_loop)
    time.sleep(60)

  5%|▍         | 27/559 [40:28<13:17:24, 89.93s/it] 


TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded